# One-Hot Encoding (원-핫 인코딩) — 기초부터 실무까지

이 노트북은 명목형(범주형) 변수를 수치형으로 변환하는 대표 기법인 'One-Hot Encoding'을 이론과 코드 예제로 
직접 실행하며 배우도록 구성했습니다. pandas, scikit-learn 예제를 포함합니다.

## 학습 목표 (Checklist)
- One-Hot Encoding의 개념과 목적 이해
- pandas의 `get_dummies` 사용법 숙지
- scikit-learn의 `OneHotEncoder` 사용법(희소행렬, handle_unknown, drop 등) 이해
- ColumnTransformer와 Pipeline에서의 적용 방법 실습
- 다중공선성, 고차원 범주, unseen category 처리 같은 실무 이슈와 대처법 학습

## 핵심 개념 요약
- 범주형 변수를 수치형으로 바꿔 모델이 다룰 수 있게 한다.
- 각 범주를 이진 변수(0/1)로 변환: 예) 색상 {red, blue, green} -> color_red, color_blue, color_green.
- 장점: 의미 보존(순서 정보가 없는 범주에 적절), 많은 모델(특히 선형 모델)에 유용.
- 단점: 변수 수 폭발(카디널리티가 높을 때), 다중공선성(모든 더미를 사용하면 완전공선성 발생), 희소 표현과 메모리 문제.

In [ ]:
# 필요한 라이브러리 로드
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

print('libraries loaded')

## 간단한 데이터셋 생성 — 명목형 컬럼 예시
실습용으로 작은 데이터프레임을 만들어 `get_dummies`와 `OneHotEncoder`를 비교합니다.

In [ ]:
df = pd.DataFrame({
    'color': ['red', 'blue', 'green', 'blue', 'red'],
    'size': ['S', 'M', 'L', 'M', 'S'],
    'price': [10, 15, 20, 15, 12]
})

df

## pandas: get_dummies 사용법
- 가장 간단한 방법. DataFrame을 그대로 받아 새로운 더미 컬럼을 만들어 반환한다.
- `drop_first=True`로 한 컬럼을 제거해 회귀 계열의 다중공선성을 완화할 수 있다.

In [ ]:
# 기본 사용
pd.get_dummies(df, columns=['color', 'size'])

# drop_first=True 예시(참고)
pd.get_dummies(df, columns=['color', 'size'], drop_first=True)

## scikit-learn: OneHotEncoder
- sklearn의 인코더는 파이프라인에 쉽게 결합된다.
- 주요 옵션: `sparse`(희소행렬 출력 여부), `handle_unknown`(훈련에 없는 범주 처리), `drop`(첫 카테고리 제거) 등.

In [ ]:
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')  # 작은 예제이므로 dense로 설정
X = df[['color', 'size']]
ohe.fit(X)
ohe.transform(X)

# feature 이름 확인(0.24+ 에서 get_feature_names_out 사용)
try:
    print('feature names:', ohe.get_feature_names_out(['color','size']))
except Exception:
    print('sklearn version may not support get_feature_names_out; use older get_feature_names if needed')

## ColumnTransformer와 Pipeline에서 범주형 처리하기
실무에서는 수치형/범주형 전처리를 분리하여 ColumnTransformer로 묶어 파이프라인에 넣는 것이 추천됩니다.

In [ ]:
# 간단한 파이프라인 예시: OneHotEncoding + LogisticRegression(데모 목적)
# 여기서는 price>12 를 타깃으로 간단한 분류 문제를 만들어 테스트합니다.
df2 = df.copy()
df2['target'] = (df2['price'] > 12).astype(int)
X = df2[['color','size']]
y = df2['target']

preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), ['color','size'])],
    remainder='drop'
)

clf = Pipeline([('pre', preprocessor), ('clf', LogisticRegression())])
clf.fit(X, y)
pred = clf.predict(X)
print('accuracy (train):', accuracy_score(y, pred))

## drop='first' 와 다중공선성
- 모든 더미 컬럼을 사용하면 sum(더미들)=1 관계 때문에 완전공선성(완벽한 상관)이 생긴다.
- 선형/로지스틱 모델에서 이를 피하려면 `drop='first'` 또는 `drop_first=True`를 사용해 하나의 카테고리를 제거한다.

In [ ]:
# drop='first' 비교 (OneHotEncoder의 drop 사용 예)
ohe_all = OneHotEncoder(sparse=False, handle_unknown='ignore', drop=None)
ohe_drop = OneHotEncoder(sparse=False, handle_unknown='ignore', drop='first')
X = df[['color','size']]
print('all dims:', ohe_all.fit_transform(X).shape)
print('drop first dims:', ohe_drop.fit_transform(X).shape)

## 실제 운영/실무 고려사항 (요약)
- 카디널리티가 매우 높은 범주(예: 수십만 user id): one-hot은 불가능에 가깝다 -> hashing, target encoding 사용 검토.
- 훈련 시 없는 새로운 범주가 운영에서 들어올 수 있음 -> `handle_unknown='ignore'` 권장 또는 새 카테고리 묶음(예: 'OTHER') 처리가 필요.
- 희소행렬을 사용하면 메모리 절약 가능(sparse=True). 일부 모델이 희소 행렬을 직접 받아들임.
- 선형 모델에서는 drop 첫 카테고리를 통해 다중공선성 완화. 트리 기반 모델은 원-핫을 써도 큰 문제가 되지 않음(트리 자체가 범주를 분할).

## 고카디널리티 대안 간단 시연: FeatureHasher(해싱 트릭)
- FeatureHasher는 텍스트나 범주형의 빠른 해싱 기반 인코딩을 제공.
- 충돌(hash collision)이 발생할 수 있으므로 결과를 해석하기 어렵다.

In [ ]:
# 해싱 예시 (범주를 문자열로 변환하여 FeatureHasher 적용)
fh = FeatureHasher(n_features=8, input_type='string')
cats = df['color'].astype(str).tolist()
X_hash = fh.transform(cats)  # 희소행렬 반환
print('hashed shape:', X_hash.shape)
print(X_hash.toarray())

## 실전 팁(마무리)
- 빠른 실험: `pd.get_dummies`로 시작해 모델 성능과 리소스 사용량을 확인.
- 프로덕션: ColumnTransformer + Pipeline으로 재현 가능 파이프라인 구성.
- unseen 카테고리 대비: `handle_unknown='ignore'` 또는 validation/feature engineering 단계에서 'OTHER'로 묶기.
- 고카디널리티: 해싱 또는 타깃 인코딩(통계적 방법) 고려.

---
만약 셀을 바로 실행해 결과 확인을 원하면 알려주세요. 필요하면 각 예제에 대한 더 깊은 해설(수학적 배경, 확률적 처리 방법, 성능 비교 표)도 추가하겠습니다.